In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/ilp2021f/train_final.csv')
train['income>50K'] = train['income>50K'].astype(np.uint8)
test = pd.read_csv('/kaggle/input/ilp2021f/test_final.csv')

In [ ]:
train.loc[train['workclass'] == '?', 'workclass'] = train['workclass'].value_counts().idxmax()
train.loc[train['occupation'] == '?', 'occupation'] = train['occupation'].value_counts().idxmax()
train.loc[train['native.country'] == '?', 'native.country'] = train['native.country'].value_counts().idxmax()

test.loc[test['workclass'] == '?', 'workclass'] = test['workclass'].value_counts().idxmax()
test.loc[test['occupation'] == '?', 'occupation'] = test['occupation'].value_counts().idxmax()
test.loc[test['native.country'] == '?', 'native.country'] = test['native.country'].value_counts().idxmax()

# print(sorted(train['native.country'].unique()))
# print(sorted(test['native.country'].unique()))

for numeric in train.select_dtypes(exclude='number'):
    encoded_columns = pd.get_dummies(train[numeric])
    train = train.join(encoded_columns).drop(numeric, axis=1)   
    
for numeric in test.select_dtypes(exclude='number'):
    encoded_columns = pd.get_dummies(test[numeric])
    test = test.join(encoded_columns).drop(numeric, axis=1)   

le = preprocessing.LabelEncoder()
for non_numeric in train.select_dtypes(exclude='number'):
    train[non_numeric] = le.fit_transform(train[non_numeric].values)
for non_numeric in test.select_dtypes(exclude='number'):
    test[non_numeric] = le.fit_transform(test[non_numeric].values)

test.drop(columns=['ID'], inplace=True)
test.drop(columns=['Holand-Netherlands'], inplace=True)

In [ ]:
# feature engineering

# fill in categorical missing values with most common value
#for non_numeric in train.select_dtypes(object):


#Dropping the outlier rows with standard deviation
# factor = 3
# for numeric in train.select_dtypes(include='number'):
#     upper_lim = train[numeric].mean () + train[numeric].std () * factor
#     lower_lim = train[numeric].mean () - train[numeric].std () * factor

#     train = train[(train[numeric] < upper_lim) & (train[numeric] > lower_lim)]

train = train[(train['capital.gain'] < 50000)]
train = train[(train['capital.loss'] < 2000)]

 

trainX = train.drop(columns=['income>50K'])
trainY = train[['income>50K']]

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size=test_size, random_state=seed)

xgb = XGBClassifier()
xgb = xgb.fit(X_train, Y_train.values.ravel())

predictions = xgb.predict(test)

submission = pd.DataFrame({'ID': range(1,23843), 'Prediction': predictions})
print(submission.info())
submission.to_csv('xgb_feature_eng8.csv', index=False)